# ROHAN — Quickstart Notebook

Interactive walkthrough of the ROHAN framework: from running a baseline simulation,
through manual strategy injection, to the full autonomous LLM-driven refinement loop.

**Sections:**
1. Setup & Configuration
2. Baseline Simulation (no strategy)
3. Manual Strategy Pipeline (validate → execute → analyse)
4. Autonomous Refinement Loop (LangGraph)
5. Inspecting Results

---

## 1. Setup & Configuration

Ensure your `.env` file is set up with the required API key at the project root:
```
OPENROUTER_API_KEY=sk-or-...
```

The default provider is **OpenRouter**, which routes to Claude (codegen) and Gemini (analysis/judge).
You can override models via `LLM_CODEGEN_MODEL`, `LLM_ANALYSIS_MODEL`, `LLM_JUDGE_MODEL`.

In [ ]:
# Core imports
import logging
import warnings

import matplotlib.pyplot as plt
import pandas as pd

# ROHAN imports
from rohan.config import SimulationSettings
from rohan.config.agent_settings import (
    AgentSettings,
    MomentumAgentSettings,
    NoiseAgentSettings,
    ValueAgentSettings,
)
from rohan.framework.analysis_service import AnalysisService
from rohan.framework.iteration_pipeline import IterationPipeline, PipelineConfig
from rohan.simulation.simulation_service import SimulationService
from rohan.simulation.strategy_validator import StrategyValidator, execute_strategy_safely
from rohan.ui.utils.presets import get_preset_config, get_preset_names

# Notebook settings
warnings.filterwarnings("ignore", category=FutureWarning)
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(name)s] %(levelname)s: %(message)s", datefmt="%H:%M:%S")

%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 5)

print("✅ Imports OK")

### 1.1 Simulation Presets

ROHAN ships with pre-configured market scenarios. Let's inspect the available presets
and pick one for testing.

In [ ]:
# List available presets
for name in get_preset_names():
    print(f"  • {name}")

# Load the default balanced-market preset
settings = get_preset_config("Default (Balanced Market)")
settings.seed = 42
print("\nUsing: Default (Balanced Market)")
print(f"  Date:  {settings.date}")
print(f"  Time:  {settings.start_time} → {settings.end_time}")
print(f"  Seed:  {settings.seed}")
print(f"  Cash:  ${settings.starting_cash / 100:,.2f}")

---
## 2. Baseline Simulation (No Strategy)

Run ABIDES with the built-in agents only (noise traders, value agents, market makers).
This gives us a baseline to compare against later.

In [ ]:
# Run baseline simulation
service = SimulationService()
baseline_result = service.run_simulation(settings, strategy=None)

if baseline_result.error:
    print(f"❌ Baseline failed: {baseline_result.error}")
else:
    print(f"✅ Baseline completed in {baseline_result.duration_seconds:.1f}s")
    baseline_output = baseline_result.result

In [ ]:
# Compute market-wide metrics for the baseline
assert baseline_output is not None
analyzer = AnalysisService()
baseline_metrics = analyzer.compute_metrics(baseline_output)

print("Baseline Market Metrics")
print("─" * 40)
print(f"  Volatility (ann.):    {baseline_metrics.volatility:.4f}" if baseline_metrics.volatility else "  Volatility: N/A")
print(f"  Mean spread:          {baseline_metrics.mean_spread:.1f} ¢" if baseline_metrics.mean_spread else "  Mean spread: N/A")
print(f"  Avg bid liquidity:    {baseline_metrics.avg_bid_liquidity:.1f}" if baseline_metrics.avg_bid_liquidity else "  Avg bid liquidity: N/A")
print(f"  Avg ask liquidity:    {baseline_metrics.avg_ask_liquidity:.1f}" if baseline_metrics.avg_ask_liquidity else "  Avg ask liquidity: N/A")
print(f"  Traded volume:        {baseline_metrics.traded_volume:,}" if baseline_metrics.traded_volume else "  Traded volume: N/A")

In [ ]:
# Visualise baseline: price series, spread, and volume
fig_price = analyzer.plot_price_series(baseline_output, title="Baseline — Price Series")
plt.show()

fig_spread = analyzer.plot_spread(baseline_output, title="Baseline — Bid-Ask Spread")
plt.show()

fig_volume = analyzer.plot_volume(baseline_output, title="Baseline — Volume at BBO")
plt.show()

### 2.1 Inspect the Order Book

The `SimulationOutput` abstraction provides typed DataFrames (validated by Pandera).

In [ ]:
# L1 order book snapshots (top-of-book)
l1 = baseline_output.get_order_book_l1()
print(f"L1 snapshots: {len(l1):,} rows")
l1.head(10)

In [ ]:
# L2 order book (depth=5 by default)
l2 = baseline_output.get_order_book_l2(n_levels=5)
print(f"L2 snapshots: {len(l2):,} rows")
l2.head(10)

---
## 3. Manual Strategy Pipeline

Before using the autonomous LLM loop, let's manually walk through the pipeline:
1. Write a simple strategy implementing the `StrategicAgent` protocol
2. Validate it (AST safety checks)
3. Execute it against the simulation
4. Analyse the results vs. baseline

### 3.1 Write a Strategy

The `StrategicAgent` protocol requires three methods:
- `initialize(config: AgentConfig)` — called once at start
- `on_market_data(state: MarketState) -> list[OrderAction]` — called on each market update
- `on_order_update(update: Order) -> list[OrderAction]` — called on fills/cancellations

All prices are in **integer cents** (e.g., `10050` = $100.50).

In [ ]:
SAMPLE_STRATEGY = '''
from rohan.simulation.models.strategy_api import (
    AgentConfig, MarketState, Order, OrderAction, OrderStatus, OrderType, Side
)


class SimpleSpreadTrader:
    """A minimal market-making strategy that places limit orders around the mid price.

    Places a bid below mid and an ask above mid, aiming to capture the spread.
    Keeps inventory bounded by skipping the side that would increase exposure.
    Cancels all stale orders before placing new quotes each tick.
    """

    def initialize(self, config: AgentConfig) -> None:
        self.max_inventory = 10
        self.spread_offset = 1  # 1 cent — must be competitive with market spread (~2¢)
        self.order_size = 1
        self.inventory = 0

    def on_tick(self, state: MarketState) -> list[OrderAction]:
        """Called on periodic wakeups. Not used for this reactive strategy."""
        return []

    def on_market_data(self, state: MarketState) -> list[OrderAction]:
        actions = []

        # 1. Cancel all existing orders to refresh our quotes
        if state.open_orders:
            actions.append(OrderAction.cancel_all())

        if state.best_bid is None or state.best_ask is None:
            return actions

        mid = (state.best_bid + state.best_ask) // 2

        # 2. Place new quotes
        # Place bid if not too long
        if self.inventory < self.max_inventory:
            actions.append(OrderAction(
                side=Side.BID,
                order_type=OrderType.LIMIT,
                quantity=self.order_size,
                price=mid - self.spread_offset,
            ))

        # Place ask if not too short
        if self.inventory > -self.max_inventory:
            actions.append(OrderAction(
                side=Side.ASK,
                order_type=OrderType.LIMIT,
                quantity=self.order_size,
                price=mid + self.spread_offset,
            ))

        return actions

    def on_order_update(self, update: Order) -> list[OrderAction]:
        # Track inventory from fills
        if update.status in (OrderStatus.FILLED, OrderStatus.PARTIAL):
            if update.side == Side.BID:
                self.inventory += update.quantity
            else:
                self.inventory -= update.quantity
        return []

    def on_simulation_end(self, final_state: MarketState) -> None:
        pass
'''

print("Loaded sample strategy")

### 3.2 Validate the Strategy

In [ ]:
validator = StrategyValidator()
validation = validator.validate(SAMPLE_STRATEGY)

if validation.is_valid:
    print("✅ Strategy passed AST validation")
else:
    print(f"❌ Validation failed: {validation.errors}")

In [ ]:
# Compile and verify the class can be instantiated
strategy_class = validator.execute_strategy(SAMPLE_STRATEGY, "SimpleSpreadTrader")
instance = strategy_class()
print(f"✅ Instantiated: {strategy_class.__name__}")
print(f"   Methods: {[m for m in dir(instance) if not m.startswith('_')]}")

### 3.3 Execute the Strategy

In [ ]:
# Run the strategy through the simulation
strategy_result = execute_strategy_safely(SAMPLE_STRATEGY, settings)

if strategy_result.error:
    print(f"❌ Strategy execution failed: {strategy_result.error}")
else:
    print(f"✅ Strategy completed in {strategy_result.duration_seconds:.1f}s")
    strategy_output = strategy_result.result

### 3.4 Analyse Results vs. Baseline

In [ ]:
# Compute strategy market metrics and agent-level metrics
assert strategy_output is not None
strategy_market_metrics = analyzer.compute_metrics(strategy_output)

assert strategy_output.strategic_agent_id is not None, "No strategic agent in output"
agent_metrics = analyzer.compute_agent_metrics(
    strategy_output,
    strategy_output.strategic_agent_id,
    initial_cash=settings.starting_cash,
)

# Print strategy agent performance
print(f"Strategic Agent ID: {strategy_output.strategic_agent_id}")
print()
print("Strategy Agent Performance")
print("─" * 40)
print(f"  PnL:              ${agent_metrics.total_pnl / 100:,.2f}" if agent_metrics.total_pnl is not None else "  PnL: N/A")
print(f"  Sharpe ratio:     {agent_metrics.sharpe_ratio:.3f}" if agent_metrics.sharpe_ratio else "  Sharpe ratio: N/A")
print(f"  Max drawdown:     ${agent_metrics.max_drawdown / 100:,.2f}" if agent_metrics.max_drawdown else "  Max drawdown: N/A")
print(f"  Trade count:      {agent_metrics.trade_count}")
print(f"  Fill rate:        {agent_metrics.fill_rate:.1%}" if agent_metrics.fill_rate else "  Fill rate: N/A")
print(f"  End inventory:    {agent_metrics.end_inventory}")

In [ ]:
# Compare market impact: how did our strategy affect the market?
def _pct(a, b):
    if a is None or b is None or b == 0:
        return None
    return (a - b) / b


impact_data = {
    "Metric": ["Volatility", "Mean Spread", "Bid Liquidity", "Ask Liquidity"],
    "Baseline": [
        baseline_metrics.volatility,
        baseline_metrics.mean_spread,
        baseline_metrics.avg_bid_liquidity,
        baseline_metrics.avg_ask_liquidity,
    ],
    "With Strategy": [
        strategy_market_metrics.volatility,
        strategy_market_metrics.mean_spread,
        strategy_market_metrics.avg_bid_liquidity,
        strategy_market_metrics.avg_ask_liquidity,
    ],
}

impact_df = pd.DataFrame(impact_data)
impact_df["Δ %"] = [f"{_pct(s, b) * 100:+.1f}%" if _pct(s, b) is not None else "N/A" for s, b in zip(impact_df["With Strategy"], impact_df["Baseline"], strict=False)]
impact_df

In [ ]:
# Strategy charts
fig_price = analyzer.plot_price_series(strategy_output, title="With Strategy — Price Series")
plt.show()

fig_spread = analyzer.plot_spread(strategy_output, title="With Strategy — Bid-Ask Spread")
plt.show()

### 3.5 Full Pipeline (validate → execute → compare → interpret)

The `IterationPipeline` wires all steps together and produces a structured
`IterationResult` including the LLM-ready interpreter prompt.

In [ ]:
pipeline = IterationPipeline()
config = PipelineConfig(
    settings=settings,
    goal_description="Create a spread-capturing market-making strategy",
    persist=False,  # no DB needed for local testing
)

result = pipeline.run(SAMPLE_STRATEGY, config, generation_number=1)

print(f"Success: {result.success}")
print(f"Duration: {result.duration_seconds:.1f}s")
if result.error:
    print(f"Error: {result.error}")

In [ ]:
# The interpreter prompt is what gets sent to the LLM for analysis
if result.interpreter_prompt:
    print(result.interpreter_prompt[:2000])
    if len(result.interpreter_prompt) > 2000:
        print(f"\n... ({len(result.interpreter_prompt) - 2000} more chars)")

---
## 4. Autonomous Refinement Loop (LangGraph)

This is the core of ROHAN: the LLM-driven strategy refinement loop.

The graph runs:
**Writer** → **Validator** → **Scenario Executor** → **Explainer** → **Aggregator** → (loop or stop)

> ⚠️ **Requires** `OPENROUTER_API_KEY` (or equivalent) in your `.env` file.

### 4.1 Visualise the Graph

In [ ]:
from rohan.llm.graph import build_refinement_graph

graph = build_refinement_graph()

# Print the graph structure
try:
    from IPython.display import Image, display

    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # Fallback: print node/edge structure
    print("Graph nodes:", list(graph.get_graph().nodes.keys()))
    for edge in graph.get_graph().edges:
        print(f"  {edge}")

### 4.2 Run the Refinement Loop

Choose a goal and let the agents iterate. Each iteration:
1. **Writer** generates a strategy
2. **Validator** checks safety (AST + instantiation)
3. **Executor** runs the strategy across scenarios
4. **Explainer** analyses each scenario with tool access
5. **Aggregator** judges convergence and provides unified feedback

Convergence: score ≥ 8 or plateau detected → stop.

In [ ]:
from rohan.llm.graph import run_refinement

# Configure the refinement run
GOAL = "Create a market-making strategy that captures the bid-ask spread while keeping inventory risk low."
MAX_ITERATIONS = 3

# You can also test across multiple scenarios:
# scenarios = [
#     ScenarioConfig(name="default"),
#     ScenarioConfig(name="volatile", config_override={"agents": {"noise": {"num_agents": 1500}}}),
# ]

print(f"Goal: {GOAL}")
print(f"Max iterations: {MAX_ITERATIONS}")
print("Starting refinement loop...\n")

final_state = run_refinement(
    goal=GOAL,
    max_iterations=MAX_ITERATIONS,
)

---
## 5. Inspecting Results

In [ ]:
# Summary
iterations = final_state.get("iterations", [])
print(f"Completed {len(iterations)} iteration(s)")
print(f"Final status: {final_state.get('status', 'unknown')}")
print()

In [ ]:
# Iteration history table
if iterations:
    history = []
    for it in iterations:
        # Collect metrics from the first (or only) scenario
        scenario_names = list(it.scenario_metrics.keys())
        first_scenario = it.scenario_metrics.get(scenario_names[0]) if scenario_names else None

        history.append(
            {
                "Iteration": it.iteration_number,
                "Score": f"{it.judge_score:.1f}/10" if it.judge_score else "N/A",
                "PnL ($)": f"{first_scenario.total_pnl / 100:,.2f}" if first_scenario and first_scenario.total_pnl else "N/A",
                "Sharpe": f"{first_scenario.sharpe_ratio:.3f}" if first_scenario and first_scenario.sharpe_ratio else "N/A",
                "Trades": first_scenario.trade_count if first_scenario else 0,
                "Vol Δ%": f"{first_scenario.volatility_delta_pct * 100:+.1f}%" if first_scenario and first_scenario.volatility_delta_pct else "N/A",
                "Spread Δ%": f"{first_scenario.spread_delta_pct * 100:+.1f}%" if first_scenario and first_scenario.spread_delta_pct else "N/A",
            }
        )

    pd.DataFrame(history).set_index("Iteration")
else:
    print("No iterations completed — check validation errors above.")

In [ ]:
# Score progression
if len(iterations) > 1:
    scores = [it.judge_score for it in iterations if it.judge_score is not None]
    if scores:
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.plot(range(1, len(scores) + 1), scores, "o-", linewidth=2, markersize=8)
        ax.set_xlabel("Iteration")
        ax.set_ylabel("Judge Score")
        ax.set_title("Strategy Quality Over Iterations")
        ax.set_ylim(0, 10.5)
        ax.set_xticks(range(1, len(scores) + 1))
        ax.grid(True, alpha=0.3)
        ax.axhline(y=8, color="green", linestyle="--", alpha=0.5, label="Convergence threshold")
        ax.legend()
        plt.tight_layout()
        plt.show()

In [ ]:
# Final strategy code
final_code = final_state.get("current_code")
if final_code:
    print("Final Strategy Code")
    print("═" * 60)
    print(final_code)
else:
    print("No strategy code produced.")
    errors = final_state.get("validation_errors", [])
    if errors:
        print(f"Validation errors: {errors}")

In [ ]:
# Judge reasoning for each iteration
for it in iterations:
    print(f"\n{'─' * 60}")
    print(f"Iteration {it.iteration_number} — Score: {it.judge_score}/10")
    print(f"{'─' * 60}")
    print(it.judge_reasoning or "(no reasoning)")

### 5.1 Re-run the Final Strategy

Take the final strategy from the refinement loop and run it through the full pipeline
to get detailed charts and metrics.

In [ ]:
if final_code:
    # Run through the full IterationPipeline
    final_pipeline = IterationPipeline()
    final_config = PipelineConfig(
        settings=settings,
        goal_description=GOAL,
        persist=False,
    )
    final_result = final_pipeline.run(final_code, final_config)

    if final_result.success and final_result.comparison:
        comp = final_result.comparison
        sm = comp.strategy_metrics
        mi = comp.market_impact

        print("Final Strategy — Detailed Results")
        print("═" * 50)
        print(f"  PnL:             ${sm.total_pnl / 100:,.2f}" if sm.total_pnl else "  PnL: N/A")
        print(f"  Trades:          {sm.trade_count}")
        print(f"  Fill rate:       {sm.fill_rate:.1%}" if sm.fill_rate else "  Fill rate: N/A")
        print(f"  End inventory:   {sm.end_inventory}")
        print()
        print("Market Impact:")
        print(f"  Spread Δ:        {mi.spread_delta_pct * 100:+.1f}%" if mi.spread_delta_pct else "  Spread Δ: N/A")
        print(f"  Volatility Δ:    {mi.volatility_delta_pct * 100:+.1f}%" if mi.volatility_delta_pct else "  Volatility Δ: N/A")
        print(f"  Bid Liq. Δ:      {mi.bid_liquidity_delta_pct * 100:+.1f}%" if mi.bid_liquidity_delta_pct else "  Bid Liq. Δ: N/A")
        print(f"  Ask Liq. Δ:      {mi.ask_liquidity_delta_pct * 100:+.1f}%" if mi.ask_liquidity_delta_pct else "  Ask Liq. Δ: N/A")
    else:
        print(f"Re-run failed: {final_result.error}")
else:
    print("No final strategy to re-run.")

In [ ]:
# Plot the final strategy run
if final_code and final_result.success:
    final_strat_result = execute_strategy_safely(final_code, settings)
    if not final_strat_result.error and final_strat_result.result:
        final_output = final_strat_result.result

        fig = analyzer.plot_price_series(final_output, title="Final Strategy — Price Series")
        plt.show()

        fig = analyzer.plot_spread(final_output, title="Final Strategy — Spread")
        plt.show()

        fig = analyzer.plot_volume(final_output, title="Final Strategy — Volume at BBO")
        plt.show()

---
## 6. Experimenting with Different Scenarios

Try different market conditions using presets or custom configurations.

In [ ]:
# Example: High Volatility preset
volatile_settings = get_preset_config("High Volatility")

print("High Volatility Configuration:")
print(f"  Oracle fund_vol:       {volatile_settings.agents.oracle.fund_vol}")
print(f"  Noise agents:          {volatile_settings.agents.noise.num_agents}")
print(f"  Momentum agents:       {volatile_settings.agents.momentum.num_agents}")
print(f"  Momentum max_size:     {volatile_settings.agents.momentum.max_size}")

In [ ]:
# Custom configuration from scratch
custom_settings = SimulationSettings(
    date="20260130",
    start_time="09:30:00",
    end_time="10:00:00",
    seed=42,  # Fixed seed for reproducibility
    starting_cash=10_000_000,
    log_orders=True,
    agents=AgentSettings(
        noise=NoiseAgentSettings(num_agents=1000),
        value=ValueAgentSettings(
            num_agents=80,
            r_bar=100_000,  # $1000 fundamental value
        ),
        momentum=MomentumAgentSettings(
            num_agents=15,
            max_size=10,
            wake_up_freq="30s",
        ),
    ),
)

print(
    f"Custom config — seed={custom_settings.seed}, agents: "
    f"noise={custom_settings.agents.noise.num_agents}, "
    f"value={custom_settings.agents.value.num_agents}, "
    f"momentum={custom_settings.agents.momentum.num_agents}"
)

---

## Next Steps

- **Streamlit UI**: `uv run ui` for a full interactive dashboard
- **CLI**: `python -m rohan.llm.cli --goal "..." --max-iterations 5`
- **Multi-scenario**: Pass multiple `ScenarioConfig` objects to `run_refinement()` for robustness testing
- **Custom presets**: Edit `src/rohan/ui/utils/presets.py` to add your own market scenarios